In [28]:
import pandas as pd
import spacy
from nltk.stem import PorterStemmer
import networkx as nx

In [ ]:
relationships_train = pd.read_csv('/workspaces/master_thesis/poincare/relationships_train_1.csv', on_bad_lines='skip', sep='\t')
relationships_test = pd.read_csv('/workspaces/master_thesis/poincare/relationships_test_1.csv', on_bad_lines='skip', sep='\t')

In [9]:
#combine train and test
relationships = pd.concat([relationships_train, relationships_test])

In [10]:
relationships

,0,1
0,Primary squamous cell carcinoma of lingual tonsil,Carcinoma of pharynx
1,Secondary malignant neoplasm of soft tissues o...,Secondary malignant neoplasm of upper limb
2,Entire prepericardial lymph node,Entire lymph node of thorax
3,Sigvaris Style Transparent (15-20mmHg) tights ...,NHS dm+d actual appliance pack
4,Congenital absence of scapula,Congenital anomaly of bone of shoulder girdle
...,...,...
222356,Smear for epithelial cells,Smear for cell identification
222357,Asclepias fructicosa,Genus Asclepias
222358,Altiven class 2 (23-32mmHg) thigh length open ...,NHS dm+d actual appliance
222359,Deposition of coal and silica,Deposition of silica


In [21]:
nlp = spacy.load("en_core_web_sm")
def preprocessing(sample):
    if isinstance(sample, str):
        sample = sample.lower()
        stemmer = PorterStemmer()
        token_list = []
        doc = nlp(sample)
        token_list = [stemmer.stem(token.text)
            for token in doc
                if not token.is_stop and not token.is_punct
            ]
        sentence = " ".join(token_list)
        return sentence
    else:
        print(f"Invalid input {sample}: expected a string.")
        return None

In [14]:
#rename columns to concept_1, concept_2
relationships = relationships.rename(columns={'0': 'concept_1', '1': 'concept_2'})

In [15]:
relationships

,concept_1,concept_2
0,Primary squamous cell carcinoma of lingual tonsil,Carcinoma of pharynx
1,Secondary malignant neoplasm of soft tissues o...,Secondary malignant neoplasm of upper limb
2,Entire prepericardial lymph node,Entire lymph node of thorax
3,Sigvaris Style Transparent (15-20mmHg) tights ...,NHS dm+d actual appliance pack
4,Congenital absence of scapula,Congenital anomaly of bone of shoulder girdle
...,...,...
222356,Smear for epithelial cells,Smear for cell identification
222357,Asclepias fructicosa,Genus Asclepias
222358,Altiven class 2 (23-32mmHg) thigh length open ...,NHS dm+d actual appliance
222359,Deposition of coal and silica,Deposition of silica


In [22]:
#use preprocessing function to preprocess relationships through the loop
relationships['concept_1'] = relationships['concept_1'].apply(preprocessing)

Invalid input nan: expected a string.
Invalid input nan: expected a string.


In [23]:
relationships['concept_2'] = relationships['concept_2'].apply(preprocessing)

In [24]:
relationships_df=relationships 

,concept_1,concept_2
0,primari squamou cell carcinoma lingual tonsil,carcinoma pharynx
1,secondari malign neoplasm soft tissu upper limb,secondari malign neoplasm upper limb
2,entir prepericardi lymph node,entir lymph node thorax
3,sigvari style transpar 15 20mmhg tight close t...,nh dm+d actual applianc pack
4,congenit absenc scapula,congenit anomali bone shoulder girdl
...,...,...
222356,smear epitheli cell,smear cell identif
222357,asclepia fructicosa,genu asclepia
222358,altiven class 2 23 32mmhg thigh length open to...,nh dm+d actual applianc
222359,deposit coal silica,deposit silica


In [33]:
#delete string with none nan values
relationships_df = relationships_df.dropna()

In [35]:
# save preprocessed relationships to a file 
relationships_df.to_csv('/workspaces/master_thesis/poincare/relationships_preprocessed.csv', index=False)

In [2]:
import pandas as pd
pd.read_csv('/workspaces/master_thesis/poincare/relationships_preprocessed.csv')

,0,1
0,primari squamou cell carcinoma lingual tonsil,carcinoma pharynx
1,secondari malign neoplasm soft tissu upper limb,secondari malign neoplasm upper limb
2,entir prepericardi lymph node,entir lymph node thorax
3,sigvari style transpar 15 20mmhg tight close t...,nh dm+d actual applianc pack
4,congenit absenc scapula,congenit anomali bone shoulder girdl
...,...,...
1111796,smear epitheli cell,smear cell identif
1111797,asclepia fructicosa,genu asclepia
1111798,altiven class 2 23 32mmhg thigh length open to...,nh dm+d actual applianc
1111799,deposit coal silica,deposit silica


In [36]:
# relationships dataframe to list of tuples
relationships = [tuple(x) for x in relationships_df.values]

In [37]:
# relationships list of tuples to networkx graph
G = nx.Graph()
G.add_edges_from(relationships)

In [41]:
from node2vec import Node2Vec

# Assuming G is your graph data structure
node2vec = Node2Vec(G, dimensions=128, walk_length=20, num_walks=200, workers=4)  # Use workers=1, quiet=True for less verbose output
model = node2vec.fit(window=10, min_count=1)


/home/vscode/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Computing transition probabilities:  41%|████      | 287737/698036 [4:29:20<261:32:53,  2.29s/it]

: 

: 

In [ ]:
#save model
model.save('/workspaces/master_thesis/poincare/node2vec_model')